## Extracting CSI data from .pcap files by Nexmon

The code below is used to convert .pcap file in .csv file

In [ ]:
import numpy as np
import pandas as pd
from csiread import Nexmon
from pathlib import Path

# Configuração dos arquivos usando Path
# Defina apenas o nome do arquivo de entrada (sem o caminho completo)
input_filename = "Rasp/csi_test.pcap"
output_filename = "Rasp/csi_output.csv"

# Obtém o diretório atual do script
current_dir = Path.cwd()

# Cria os caminhos completos para os arquivos
pcap_file = current_dir / input_filename
csv_file = current_dir / output_filename

print(f"Arquivo de entrada: {pcap_file}")
print(f"Arquivo de saída: {csv_file}")
print(f"Diretório atual: {current_dir}")

# Verifica se o arquivo de entrada existe
if not pcap_file.exists():
    print(f"ERRO: Arquivo {input_filename} não encontrado no diretório atual!")
    print("Arquivos disponíveis no diretório:")
    for file in current_dir.glob("*.pcap"):
        print(f"  - {file.name}")
    exit(1)
else:
    print(f"✓ Arquivo de entrada encontrado: {input_filename}")

#Settings: chipset and bw
chipset   = "43455c0"   #Raspberry Pi 4 (BCM43455c0)
bw        = 40          #It must be the same used in the capture

#Reading .pcap file with Nexmon
print(f"Lendo arquivo: {pcap_file} ...")
# Converte Path para string para compatibilidade com csiread
reader = Nexmon(str(pcap_file), chip=chipset, bw=bw)
reader.read()

csi = reader.csi   #Expected shape: (n_packets, n_subcarriers)

print(f"Shape do CSI: {csi.shape}")

#Conversion to amplitude and phase
amplitude = np.abs(csi)
phase = np.angle(csi)

#Each line is equal to one packet and each column is a subcarrier (amplitude and phase)
df_amp = pd.DataFrame(amplitude)
df_phase = pd.DataFrame(phase)

#Rename columns
df_amp.columns   = [f"subcarrier_{i}"       for i in range(df_amp.shape[1])]
df_phase.columns = [f"subcarrier_{i}_phase" for i in range(df_phase.shape[1])]

#Concatenate: amplitude + phase
df_out = pd.concat([df_amp, df_phase], axis=1)

#Exporting to .csv file (output file)
# Converte Path para string para compatibilidade com pandas
df_out.to_csv(str(csv_file), index=False)
print(f"Arquivo CSV gerado: {csv_file}")
